In [ ]:
import warnings
import os
import cv2

import glob
import shutil
import sys
import torch
from PIL import Image
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd

from tqdm.notebook import tqdm

import ast
sys.path.append('../input/ensemble-boxes-104/ensemble_boxes-1.0.4')
from ensemble_boxes import *

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolo-arial/Arial.ttf /root/.config/Ultralytics/

# Yolov5x

In [ ]:
def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    return bboxes

def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    return bboxes

def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    return bboxes

def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    # converstion (xmid, ymid) => (xmin, ymin)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    return bboxes

def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def draw_bboxes(img, bboxes, classes, class_ids, colors = None, show_classes = None, bbox_format = 'yolo', class_name = False, line_thickness = 2):

    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors

    if bbox_format == 'yolo':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:

                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness)

    elif bbox_format == 'coco':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w  = int(round(bbox[2]))
                h  = int(round(bbox[3]))

                voc_bbox = (x1, y1, x1+w, y1+h)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)

    elif bbox_format == 'voc_pascal':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image

def show_img(img, bboxes, bbox_format='yolo', bbox_colors = None):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes,
                           classes = names,
                           class_ids = labels,
                           class_name = True,
                           colors = colors if bbox_colors is None else bbox_colors,
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((800, 400))

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]


def load_model(ckpt_path, conf=0.25, iou=0.50):
    model = torch.hub.load('/kaggle/input/yolov5',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    return model

def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    if len(bboxes):
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        classes  = preds.name.values
        return bboxes, confs,classes
    else:
        return [],[],[]

def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot



# Ensemble

In [ ]:
CKPT_PATH1 = '../input/greatbarrierreef/yolov5xexp6.pt'
CONF1      = 0.29
IOU1       = 0.40
CKPT_PATH2 = '../input/greatbarrierreef/yolov5xver275.pt'
CONF2      = 0.29
IOU2       = 0.40
CKPT_PATH3 = '../input/greatbarrierreef/yolov5xver28.pt'
CONF3      = 0.29
IOU3       = 0.40

CKPT_PATH4 = '../input/greatbarrierreef/yolov5xfold0exp9.pt'
CONF4      = 0.29
IOU4       = 0.40
CKPT_PATH5 = '../input/greatbarrierreef/yolov5xfold1exp7.pt'
CONF5      = 0.29
IOU5       = 0.40
CKPT_PATH6 = '../input/greatbarrierreef/yolov5xfold2exp5.pt'
CONF6      = 0.29
IOU6       = 0.40
CKPT_PATH7 = '../input/greatbarrierreef/yolov5xfold3exp8.pt'
CONF7      = 0.29
IOU7       = 0.40
CKPT_PATH8 = '../input/greatbarrierreef/yolov5xfold4exp1.pt'
CONF8      = 0.29
IOU8       = 0.40

CKPT_PATH9 = '../input/greatbarrierreef/fold0ver22.pt'
CONF9      = 0.29
IOU9       = 0.40
CKPT_PATH10 = '../input/greatbarrierreef/fold1ver21.pt'
CONF10      = 0.29
IOU10       = 0.40
CKPT_PATH11 = '../input/greatbarrierreef/fold2ver23.pt'
CONF11      = 0.29
IOU11       = 0.40
CKPT_PATH12 = '../input/greatbarrierreef/fold3ver14.pt'
CONF12      = 0.29
IOU12       = 0.40
CKPT_PATH13 = '../input/greatbarrierreef/fold4ver11.pt'
CONF13      = 0.29
IOU13       = 0.40

CKPT_PATH14 = '../input/greatbarrierreef/fold0ver26v.pt'
CONF14      = 0.29
IOU14       = 0.40
CKPT_PATH15 = '../input/greatbarrierreef/fold1ver25v.pt'
CONF15      = 0.29
IOU15       = 0.40
CKPT_PATH16 = '../input/greatbarrierreef/fold2ver27v.pt'
CONF16      = 0.29
IOU16       = 0.40


CKPT_PATH17 = '../input/greatbarrierreef/fold0ver19.pt'
CONF17      = 0.29
IOU17       = 0.40
CKPT_PATH18 = '../input/greatbarrierreef/fold1ver17.pt'
CONF18      = 0.29
IOU18       = 0.40
CKPT_PATH19 = '../input/greatbarrierreef/fold1ver20.pt'
CONF19      = 0.29
IOU19       = 0.40
CKPT_PATH20 = '../input/greatbarrierreef/v5x_L9_HP_FP_RO_FOLD4.pt'
CONF20      = 0.29
IOU20       = 0.40

CKPT_PATH21 = '../input/greatbarrierreef/yolov5xver29.pt'
CONF21      = 0.29
IOU21       = 0.40
CKPT_PATH22 = '../input/greatbarrierreef/yolov5xver30.pt'
CONF22      = 0.29
IOU22       = 0.40
CKPT_PATH23 = '../input/greatbarrierreef/yolov5xver31.pt'
CONF23      = 0.29
IOU23       = 0.40
CKPT_PATH24 = '../input/greatbarrierreef/yolov5xver32.pt'
CONF24      = 0.29
IOU24       = 0.40


#--------------------------------------------------------------------------
model1 = load_model(CKPT_PATH1, conf=CONF1, iou=IOU1) 
model2 = load_model(CKPT_PATH2, conf=CONF2, iou=IOU2)
model3 = load_model(CKPT_PATH3, conf=CONF3, iou=IOU3)

model4 = load_model(CKPT_PATH4, conf=CONF4, iou=IOU4) 
model5 = load_model(CKPT_PATH5, conf=CONF5, iou=IOU5)
model6 = load_model(CKPT_PATH6, conf=CONF6, iou=IOU6)
model7 = load_model(CKPT_PATH7, conf=CONF7, iou=IOU7) 
model8 = load_model(CKPT_PATH8, conf=CONF8, iou=IOU8)

model9 = load_model(CKPT_PATH9, conf=CONF9, iou=IOU9) 
model10 = load_model(CKPT_PATH10, conf=CONF10, iou=IOU10)
model11 = load_model(CKPT_PATH11, conf=CONF11, iou=IOU11)
model12 = load_model(CKPT_PATH12, conf=CONF12, iou=IOU12) 
model13 = load_model(CKPT_PATH13, conf=CONF13, iou=IOU13)

model14 = load_model(CKPT_PATH14, conf=CONF14, iou=IOU14) 
model15 = load_model(CKPT_PATH15, conf=CONF15, iou=IOU15)
model16 = load_model(CKPT_PATH16, conf=CONF16, iou=IOU16)

model17 = load_model(CKPT_PATH17, conf=CONF17, iou=IOU17) 
model18 = load_model(CKPT_PATH18, conf=CONF18, iou=IOU18)
model19 = load_model(CKPT_PATH19, conf=CONF19, iou=IOU19)
model20 = load_model(CKPT_PATH20, conf=CONF20, iou=IOU20) 

model21 = load_model(CKPT_PATH21, conf=CONF21, iou=IOU21) 
model22 = load_model(CKPT_PATH22, conf=CONF22, iou=IOU22)
model23 = load_model(CKPT_PATH23, conf=CONF23, iou=IOU23)
model24 = load_model(CKPT_PATH24, conf=CONF24, iou=IOU24) 


# Check

In [ ]:

TEST_IMAGE_PATH = "/kaggle/input/tensorflow-great-barrier-reef/train_images/video_2/5747.jpg"
img = cv2.imread(TEST_IMAGE_PATH)
#--------------------------------------------------------------------------
IMG_SIZE  = 1280
AUGMENT   = False
#--------------------------------------------------------------------------
bboxes1, scores1 ,bbclasses1  = predict(model1, img, size=IMG_SIZE, augment=AUGMENT)
bboxes2, scores2 ,bbclasses2  = predict(model2, img, size=IMG_SIZE, augment=AUGMENT)
bboxes3, scores3 ,bbclasses3  = predict(model3, img, size=IMG_SIZE, augment=AUGMENT)

bboxes4, scores4 ,bbclasses4  = predict(model4, img, size=IMG_SIZE, augment=AUGMENT)
bboxes5, scores5 ,bbclasses5  = predict(model5, img, size=IMG_SIZE, augment=AUGMENT)
bboxes6, scores6 ,bbclasses6  = predict(model6, img, size=IMG_SIZE, augment=AUGMENT)
bboxes7, scores7 ,bbclasses7  = predict(model7, img, size=IMG_SIZE, augment=AUGMENT)
bboxes8, scores8 ,bbclasses8  = predict(model8, img, size=IMG_SIZE, augment=AUGMENT)

bboxes9, scores9 ,bbclasses9  = predict(model9, img, size=IMG_SIZE, augment=AUGMENT)
bboxes10, scores10 ,bbclasses10  = predict(model10, img, size=IMG_SIZE, augment=AUGMENT)
bboxes11, scores11 ,bbclasses11  = predict(model11, img, size=IMG_SIZE, augment=AUGMENT)
bboxes12, scores12 ,bbclasses12  = predict(model12, img, size=IMG_SIZE, augment=AUGMENT)
bboxes13, scores13 ,bbclasses13  = predict(model13, img, size=IMG_SIZE, augment=AUGMENT)

bboxes14, scores14 ,bbclasses14  = predict(model14, img, size=IMG_SIZE, augment=AUGMENT)
bboxes15, scores15 ,bbclasses15  = predict(model15, img, size=IMG_SIZE, augment=AUGMENT)
bboxes16, scores16 ,bbclasses16  = predict(model16, img, size=IMG_SIZE, augment=AUGMENT)

bboxes17, scores17 ,bbclasses17  = predict(model17, img, size=IMG_SIZE, augment=AUGMENT)
bboxes18, scores18 ,bbclasses18  = predict(model18, img, size=IMG_SIZE, augment=AUGMENT)
bboxes19, scores19 ,bbclasses19  = predict(model19, img, size=IMG_SIZE, augment=AUGMENT)
bboxes20, scores20 ,bbclasses20  = predict(model20, img, size=IMG_SIZE, augment=AUGMENT)

bboxes21, scores21 ,bbclasses21  = predict(model21, img, size=IMG_SIZE, augment=AUGMENT)
bboxes22, scores22 ,bbclasses22  = predict(model22, img, size=IMG_SIZE, augment=AUGMENT)
bboxes23, scores23 ,bbclasses23  = predict(model23, img, size=IMG_SIZE, augment=AUGMENT)
bboxes24, scores24 ,bbclasses24  = predict(model24, img, size=IMG_SIZE, augment=AUGMENT)

#--------------------------------------------------------------------------
out_image=draw_bboxes(img = img,
                        bboxes = bboxes1, 
                        classes = ["model1"]*len(bboxes1),
                        class_ids = [0]*len(bboxes1),
                        class_name = True, 
                        colors = (0,0,130), 
                        bbox_format = 'coco',
                        line_thickness = 2)
out_image=draw_bboxes(img = out_image,
                        bboxes = bboxes2, 
                        classes = ["    model2"]*len(bboxes2),
                        class_ids = [0]*len(bboxes2),
                        class_name = True, 
                        colors = (0,130,0), 
                        bbox_format = 'coco',
                        line_thickness = 2)
out_image=draw_bboxes(img = out_image,
                        bboxes = bboxes3, 
                        classes = ["     model3"]*len(bboxes3),
                        class_ids = [0]*len(bboxes3),
                        class_name = True, 
                        colors = (255,0,0), 
                        bbox_format = 'coco',
                        line_thickness = 2)

out_image = cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB)
display(Image.fromarray(out_image))




bboxes1[:, 2] = bboxes1[:, 2] + bboxes1[:, 0]
bboxes1[:, 3] = bboxes1[:, 3] + bboxes1[:, 1]
bboxes2[:, 2] = bboxes2[:, 2] + bboxes2[:, 0]
bboxes2[:, 3] = bboxes2[:, 3] + bboxes2[:, 1]
bboxes3[:, 2] = bboxes3[:, 2] + bboxes3[:, 0]
bboxes3[:, 3] = bboxes3[:, 3] + bboxes3[:, 1]

bboxes4[:, 2] = bboxes4[:, 2] + bboxes4[:, 0]
bboxes4[:, 3] = bboxes4[:, 3] + bboxes4[:, 1]
bboxes5[:, 2] = bboxes5[:, 2] + bboxes5[:, 0]
bboxes5[:, 3] = bboxes5[:, 3] + bboxes5[:, 1]
bboxes6[:, 2] = bboxes6[:, 2] + bboxes6[:, 0]
bboxes6[:, 3] = bboxes6[:, 3] + bboxes6[:, 1]
bboxes7[:, 2] = bboxes7[:, 2] + bboxes7[:, 0]
bboxes7[:, 3] = bboxes7[:, 3] + bboxes7[:, 1]
bboxes8[:, 2] = bboxes8[:, 2] + bboxes8[:, 0]
bboxes8[:, 3] = bboxes8[:, 3] + bboxes8[:, 1]

bboxes9[:, 2] = bboxes9[:, 2] + bboxes9[:, 0]
bboxes9[:, 3] = bboxes9[:, 3] + bboxes9[:, 1]
bboxes10[:, 2] = bboxes10[:, 2] + bboxes10[:, 0]
bboxes10[:, 3] = bboxes10[:, 3] + bboxes10[:, 1]
bboxes11[:, 2] = bboxes11[:, 2] + bboxes11[:, 0]
bboxes11[:, 3] = bboxes11[:, 3] + bboxes11[:, 1]
bboxes12[:, 2] = bboxes12[:, 2] + bboxes12[:, 0]
bboxes12[:, 3] = bboxes12[:, 3] + bboxes12[:, 1]
bboxes13[:, 2] = bboxes13[:, 2] + bboxes13[:, 0]
bboxes13[:, 3] = bboxes13[:, 3] + bboxes13[:, 1]

bboxes14[:, 2] = bboxes14[:, 2] + bboxes14[:, 0]
bboxes14[:, 3] = bboxes14[:, 3] + bboxes14[:, 1]
bboxes15[:, 2] = bboxes15[:, 2] + bboxes15[:, 0]
bboxes15[:, 3] = bboxes15[:, 3] + bboxes15[:, 1]
bboxes16[:, 2] = bboxes16[:, 2] + bboxes16[:, 0]
bboxes16[:, 3] = bboxes16[:, 3] + bboxes16[:, 1]

bboxes17[:, 2] = bboxes17[:, 2] + bboxes17[:, 0]
bboxes17[:, 3] = bboxes17[:, 3] + bboxes17[:, 1]
bboxes18[:, 2] = bboxes18[:, 2] + bboxes18[:, 0]
bboxes18[:, 3] = bboxes18[:, 3] + bboxes18[:, 1]
bboxes19[:, 2] = bboxes19[:, 2] + bboxes19[:, 0]
bboxes19[:, 3] = bboxes19[:, 3] + bboxes19[:, 1]
bboxes20[:, 2] = bboxes20[:, 2] + bboxes20[:, 0]
bboxes20[:, 3] = bboxes20[:, 3] + bboxes20[:, 1]

bboxes21[:, 2] = bboxes21[:, 2] + bboxes21[:, 0]
bboxes21[:, 3] = bboxes21[:, 3] + bboxes21[:, 1]
bboxes22[:, 2] = bboxes22[:, 2] + bboxes22[:, 0]
bboxes22[:, 3] = bboxes22[:, 3] + bboxes22[:, 1]
bboxes23[:, 2] = bboxes23[:, 2] + bboxes23[:, 0]
bboxes23[:, 3] = bboxes23[:, 3] + bboxes23[:, 1]
bboxes24[:, 2] = bboxes24[:, 2] + bboxes24[:, 0]
bboxes24[:, 3] = bboxes24[:, 3] + bboxes24[:, 1]

# Weighted-Boxes-Fusion

In [ ]:
def run_wbf(bboxes, confs,classs, image_size, iou_thr=0.40, skip_box_thr=0.05, weights=None):
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [ ]:
#--------------------------------- Aug -----------------------------------------
iou_thr = 0.4
skip_box_thr = 0.29
#sigma = 0.01
#weights = [1, 1 ,1]
#weights = [1, 1 ,1, 1, 1, 1, 1, 1]
#weights = [1, 1 ,1, 1, 1, 1, 1, 1, 1, 1 ,1, 1, 1, 1, 1, 1]
weights = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
#--------------------------------- List -----------------------------------------
boxes_list=[bboxes1,bboxes2,bboxes3,bboxes4,bboxes5,bboxes6,bboxes7,bboxes8,bboxes9,bboxes10,bboxes11,bboxes12,bboxes13,bboxes14,bboxes15,bboxes16,bboxes17,bboxes18,bboxes19,bboxes20,bboxes21,bboxes22,bboxes23,bboxes24]
scores_list=[scores1,scores2,scores3,scores4,scores5,scores6,scores7,scores8,scores9,scores10,scores11,scores12,scores13,scores14,scores15,scores16,scores17,scores18,scores19,scores20,scores21,scores22,scores23,scores24]
labels_list=[bbclasses1,bbclasses2,bbclasses3,bbclasses4,bbclasses5,bbclasses6,bbclasses7,bbclasses8,bbclasses9,bbclasses10,bbclasses11,bbclasses12,bbclasses13,bbclasses14,bbclasses15,bbclasses16,bbclasses17,bbclasses18,bbclasses19,bbclasses20,bbclasses21,bbclasses22,bbclasses23,bbclasses24]



#####################################  WBF  #####################################
boxes, scores, labels  = run_wbf(boxes_list, scores_list,labels_list,IMG_SIZE, iou_thr, skip_box_thr,weights=weights)
boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
out_image_esemble=draw_bboxes(img = img,
                        bboxes = boxes, 
                        classes = ["WBF"]*len(boxes),
                        class_ids = [0]*len(boxes),
                        class_name = True, 
                        colors = (50,50,50), 
                        bbox_format = 'coco',
                        line_thickness = 2)
out_image_esemble = cv2.cvtColor(out_image_esemble, cv2.COLOR_BGR2RGB)
print("------------------------------  WBF   ---------------------------------------")
display(Image.fromarray(out_image_esemble))

In [ ]:
scores

# Inference

In [ ]:
%cd /kaggle/working/

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [ ]:
Method ="WBF" # NMS,Soft-NMS,NMW or WBF
IMG_SIZE  = 1280
AUGMENT   = False

iou_thr = 0.4
skip_box_thr = 0.29
#sigma = 0.01
#weights = [1, 1 ,1]
#weights = [1, 1 ,1, 1, 1, 1, 1, 1]
#weights = [1, 1 ,1, 1, 1, 1, 1, 1, 1, 1 ,1, 1, 1, 1, 1, 1]
weights = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

#--------------------------------------------------------------------------
model1 = load_model(CKPT_PATH1, conf=CONF1, iou=IOU1) 
model2 = load_model(CKPT_PATH2, conf=CONF2, iou=IOU2)
model3 = load_model(CKPT_PATH3, conf=CONF3, iou=IOU3)

model4 = load_model(CKPT_PATH4, conf=CONF4, iou=IOU4) 
model5 = load_model(CKPT_PATH5, conf=CONF5, iou=IOU5)
model6 = load_model(CKPT_PATH6, conf=CONF6, iou=IOU6)
model7 = load_model(CKPT_PATH7, conf=CONF7, iou=IOU7) 
model8 = load_model(CKPT_PATH8, conf=CONF8, iou=IOU8)

model9 = load_model(CKPT_PATH9, conf=CONF9, iou=IOU9) 
model10 = load_model(CKPT_PATH10, conf=CONF10, iou=IOU10)
model11 = load_model(CKPT_PATH11, conf=CONF11, iou=IOU11)
model12 = load_model(CKPT_PATH12, conf=CONF12, iou=IOU12) 
model13 = load_model(CKPT_PATH13, conf=CONF13, iou=IOU13)

model14 = load_model(CKPT_PATH14, conf=CONF14, iou=IOU14) 
model15 = load_model(CKPT_PATH15, conf=CONF15, iou=IOU15)
model16 = load_model(CKPT_PATH16, conf=CONF16, iou=IOU16)

model17 = load_model(CKPT_PATH17, conf=CONF17, iou=IOU17) 
model18 = load_model(CKPT_PATH18, conf=CONF18, iou=IOU18)
model19 = load_model(CKPT_PATH19, conf=CONF19, iou=IOU19)
model20 = load_model(CKPT_PATH20, conf=CONF20, iou=IOU20) 

model21 = load_model(CKPT_PATH21, conf=CONF21, iou=IOU21) 
model22 = load_model(CKPT_PATH22, conf=CONF22, iou=IOU22)
model23 = load_model(CKPT_PATH23, conf=CONF23, iou=IOU23)
model24 = load_model(CKPT_PATH24, conf=CONF24, iou=IOU24) 

#--------------------------------------------------------------------------
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    bboxes1, scores1,bbclasses1  = predict(model1, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes2, scores2,bbclasses2  = predict(model2, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes3, scores3,bbclasses3  = predict(model3, img, size=IMG_SIZE, augment=AUGMENT)
    
    bboxes4, scores4,bbclasses4  = predict(model4, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes5, scores5,bbclasses5  = predict(model5, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes6, scores6,bbclasses6  = predict(model6, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes7, scores7,bbclasses7  = predict(model7, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes8, scores8,bbclasses8  = predict(model8, img, size=IMG_SIZE, augment=AUGMENT)
    
    bboxes9, scores9 ,bbclasses9  = predict(model9, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes10, scores10 ,bbclasses10  = predict(model10, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes11, scores11 ,bbclasses11  = predict(model11, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes12, scores12 ,bbclasses12  = predict(model12, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes13, scores13 ,bbclasses13  = predict(model13, img, size=IMG_SIZE, augment=AUGMENT)
    
    bboxes14, scores14 ,bbclasses14  = predict(model14, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes15, scores15 ,bbclasses15  = predict(model15, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes16, scores16 ,bbclasses16  = predict(model16, img, size=IMG_SIZE, augment=AUGMENT)
    
    bboxes17, scores17 ,bbclasses17  = predict(model17, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes18, scores18 ,bbclasses18  = predict(model18, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes19, scores19 ,bbclasses19  = predict(model19, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes20, scores20 ,bbclasses20  = predict(model20, img, size=IMG_SIZE, augment=AUGMENT)

    bboxes21, scores21 ,bbclasses21  = predict(model21, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes22, scores22 ,bbclasses22  = predict(model22, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes23, scores23 ,bbclasses23  = predict(model23, img, size=IMG_SIZE, augment=AUGMENT)
    bboxes24, scores24 ,bbclasses24  = predict(model24, img, size=IMG_SIZE, augment=AUGMENT)
    
    
    boxes_list=[]
    scores_list=[]
    labels_list=[]
    
    count = 0
    
    
    if len(bboxes1)>0:
        boxes_list.append(bboxes1)
        scores_list.append(scores1)
        labels_list.append(bbclasses1) 
        bboxes1[:, 2] = bboxes1[:, 2] + bboxes1[:, 0]
        bboxes1[:, 3] = bboxes1[:, 3] + bboxes1[:, 1]
        count = count + 1
    if len(bboxes2)>0:
        boxes_list.append(bboxes2)
        scores_list.append(scores2)
        labels_list.append(bbclasses2)
        bboxes2[:, 2] = bboxes2[:, 2] + bboxes2[:, 0]
        bboxes2[:, 3] = bboxes2[:, 3] + bboxes2[:, 1]
        count = count + 1
    if len(bboxes3)>0:
        boxes_list.append(bboxes3)
        scores_list.append(scores3)
        labels_list.append(bbclasses3)
        bboxes3[:, 2] = bboxes3[:, 2] + bboxes3[:, 0]
        bboxes3[:, 3] = bboxes3[:, 3] + bboxes3[:, 1]
        count = count + 1
    
    
    if len(bboxes4)>0:
        boxes_list.append(bboxes4)
        scores_list.append(scores4)
        labels_list.append(bbclasses4)
        bboxes4[:, 2] = bboxes4[:, 2] + bboxes4[:, 0]
        bboxes4[:, 3] = bboxes4[:, 3] + bboxes4[:, 1]
        count = count + 1
    if len(bboxes5)>0:
        boxes_list.append(bboxes5)
        scores_list.append(scores5)
        labels_list.append(bbclasses5)
        bboxes5[:, 2] = bboxes5[:, 2] + bboxes5[:, 0]
        bboxes5[:, 3] = bboxes5[:, 3] + bboxes5[:, 1]
        count = count + 1
    if len(bboxes6)>0:
        boxes_list.append(bboxes6)
        scores_list.append(scores6)
        labels_list.append(bbclasses6)
        bboxes6[:, 2] = bboxes6[:, 2] + bboxes6[:, 0]
        bboxes6[:, 3] = bboxes6[:, 3] + bboxes6[:, 1]
        count = count + 1
    if len(bboxes7)>0:
        boxes_list.append(bboxes7)
        scores_list.append(scores7)
        labels_list.append(bbclasses7)
        bboxes7[:, 2] = bboxes7[:, 2] + bboxes7[:, 0]
        bboxes7[:, 3] = bboxes7[:, 3] + bboxes7[:, 1]
        count = count + 1
    if len(bboxes8)>0:
        boxes_list.append(bboxes8)
        scores_list.append(scores8)
        labels_list.append(bbclasses8)
        bboxes8[:, 2] = bboxes8[:, 2] + bboxes8[:, 0]
        bboxes8[:, 3] = bboxes8[:, 3] + bboxes8[:, 1]
        count = count + 1
    
    if len(bboxes9)>0:
        boxes_list.append(bboxes9)
        scores_list.append(scores9)
        labels_list.append(bbclasses9)
        bboxes9[:, 2] = bboxes9[:, 2] + bboxes9[:, 0]
        bboxes9[:, 3] = bboxes9[:, 3] + bboxes9[:, 1]
        count = count + 1
    if len(bboxes10)>0:
        boxes_list.append(bboxes10)
        scores_list.append(scores10)
        labels_list.append(bbclasses10)
        bboxes10[:, 2] = bboxes10[:, 2] + bboxes10[:, 0]
        bboxes10[:, 3] = bboxes10[:, 3] + bboxes10[:, 1]
        count = count + 1
    if len(bboxes11)>0:
        boxes_list.append(bboxes11)
        scores_list.append(scores11)
        labels_list.append(bbclasses11)
        bboxes11[:, 2] = bboxes11[:, 2] + bboxes11[:, 0]
        bboxes11[:, 3] = bboxes11[:, 3] + bboxes11[:, 1]
        count = count + 1
    if len(bboxes12)>0:
        boxes_list.append(bboxes12)
        scores_list.append(scores12)
        labels_list.append(bbclasses12)
        bboxes12[:, 2] = bboxes12[:, 2] + bboxes12[:, 0]
        bboxes12[:, 3] = bboxes12[:, 3] + bboxes12[:, 1]
        count = count + 1
    if len(bboxes13)>0:
        boxes_list.append(bboxes13)
        scores_list.append(scores13)
        labels_list.append(bbclasses13)
        bboxes13[:, 2] = bboxes13[:, 2] + bboxes13[:, 0]
        bboxes13[:, 3] = bboxes13[:, 3] + bboxes13[:, 1]
        count = count + 1
    
    if len(bboxes14)>0:
        boxes_list.append(bboxes14)
        scores_list.append(scores14)
        labels_list.append(bbclasses14) 
        bboxes14[:, 2] = bboxes14[:, 2] + bboxes14[:, 0]
        bboxes14[:, 3] = bboxes14[:, 3] + bboxes14[:, 1]
        count = count + 1
    if len(bboxes15)>0:
        boxes_list.append(bboxes15)
        scores_list.append(scores15)
        labels_list.append(bbclasses15)
        bboxes15[:, 2] = bboxes15[:, 2] + bboxes15[:, 0]
        bboxes15[:, 3] = bboxes15[:, 3] + bboxes15[:, 1]
        count = count + 1
    if len(bboxes16)>0:
        boxes_list.append(bboxes16)
        scores_list.append(scores16)
        labels_list.append(bbclasses16)
        bboxes16[:, 2] = bboxes16[:, 2] + bboxes16[:, 0]
        bboxes16[:, 3] = bboxes16[:, 3] + bboxes16[:, 1]
        count = count + 1
    
    if len(bboxes17)>0:
        boxes_list.append(bboxes17)
        scores_list.append(scores17)
        labels_list.append(bbclasses17)
        bboxes17[:, 2] = bboxes17[:, 2] + bboxes17[:, 0]
        bboxes17[:, 3] = bboxes17[:, 3] + bboxes17[:, 1]
        count = count + 1
    if len(bboxes18)>0:
        boxes_list.append(bboxes18)
        scores_list.append(scores18)
        labels_list.append(bbclasses18)
        bboxes18[:, 2] = bboxes18[:, 2] + bboxes18[:, 0]
        bboxes18[:, 3] = bboxes18[:, 3] + bboxes18[:, 1]
        count = count + 1
    if len(bboxes19)>0:
        boxes_list.append(bboxes19)
        scores_list.append(scores19)
        labels_list.append(bbclasses19)
        bboxes19[:, 2] = bboxes19[:, 2] + bboxes19[:, 0]
        bboxes19[:, 3] = bboxes19[:, 3] + bboxes19[:, 1]
        count = count + 1
    if len(bboxes20)>0:
        boxes_list.append(bboxes20)
        scores_list.append(scores20)
        labels_list.append(bbclasses20)
        bboxes20[:, 2] = bboxes20[:, 2] + bboxes20[:, 0]
        bboxes20[:, 3] = bboxes20[:, 3] + bboxes20[:, 1]
        count = count + 1
    
    if len(bboxes21)>0:
        boxes_list.append(bboxes21)
        scores_list.append(scores21)
        labels_list.append(bbclasses21)
        bboxes21[:, 2] = bboxes21[:, 2] + bboxes21[:, 0]
        bboxes21[:, 3] = bboxes21[:, 3] + bboxes21[:, 1]
        count = count + 1
    if len(bboxes22)>0:
        boxes_list.append(bboxes22)
        scores_list.append(scores22)
        labels_list.append(bbclasses22)
        bboxes22[:, 2] = bboxes22[:, 2] + bboxes22[:, 0]
        bboxes22[:, 3] = bboxes22[:, 3] + bboxes22[:, 1]
        count = count + 1
    if len(bboxes23)>0:
        boxes_list.append(bboxes23)
        scores_list.append(scores23)
        labels_list.append(bbclasses23)
        bboxes23[:, 2] = bboxes23[:, 2] + bboxes23[:, 0]
        bboxes23[:, 3] = bboxes23[:, 3] + bboxes23[:, 1]
        count = count + 1
    if len(bboxes24)>0:
        boxes_list.append(bboxes24)
        scores_list.append(scores24)
        labels_list.append(bbclasses24)
        bboxes24[:, 2] = bboxes24[:, 2] + bboxes24[:, 0]
        bboxes24[:, 3] = bboxes24[:, 3] + bboxes24[:, 1]
        count = count + 1
    

    if count>11:
    #if (len(bboxes1) + len(bboxes2) + len(bboxes3) + len(bboxes4) + len(bboxes5) + len(bboxes6) + len(bboxes7) + len(bboxes8) + len(bboxes9) + len(bboxes10) + len(bboxes11) + len(bboxes12) + len(bboxes13) + len(bboxes14) + len(bboxes15) + len(bboxes16) + len(bboxes17) + len(bboxes18) + len(bboxes19) + len(bboxes20) + len(bboxes21) + len(bboxes22) + len(bboxes23) + len(bboxes24))>0:
       
        boxes, scores, labels  = run_wbf(boxes_list, scores_list, labels_list, IMG_SIZE, iou_thr, skip_box_thr, weights=weights)

        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    else: boxes=[]
        
    
    predictions = []
    for i in range(len(boxes)):
        box = boxes[i]
        score = scores[i]
        if score > 0.05:
            x_min = int(box[0])
            y_min = int(box[1])
            bbox_width = int(box[2])
            bbox_height = int(box[3])
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    
    prediction_str = ' '.join(predictions)
    pred_df['annotations'] = prediction_str
    env.predict(pred_df)

        
    print("------------------------------  " + Method +"   ---------------------------------------")
    if idx<3:
        display(show_img(img, boxes, bbox_format='coco'))

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()